In [4]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark



Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [7]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


In [6]:
#Download the Postgres driver to connect tothe 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-05 13:41:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-05-05 13:41:25 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [8]:
 from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [10]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [7]:
data_df.count()

2643619

In [65]:
# get data regarding review
from pyspark.sql.functions import to_date
review_id_df = data_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).dropDuplicates(subset = ['review_id'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R101CJ0JBI9J43|   51202138|B00IEXBLX0|     496279633| 2015-04-09|
|R101P7GUI5O1WG|   29893430|B004ZM7IJA|     569386732| 2014-12-23|
|R102LL91IX39PK|   23424223|B001DCAAP4|     430789973| 2014-12-12|
|R102YVSWWEAP5K|   15232925|B000A7UZA8|     266396365| 2012-03-20|
|R1042WRU1WDUHY|   23868073|B0027J7AEQ|     423324239| 2012-06-05|
|R104G7VC1XPVOO|   14002185|B001EUG65I|     228112229| 2014-09-18|
|R10557FTU9GHY8|   44802822|B000OCQ6DY|     902394513| 2015-06-04|
|R105B7Y5ALJ45D|   42933060|B001GIGRRA|     136551003| 2013-09-20|
|R105C41QCCFLIN|   50514725|B0009X29WK|      83672627| 2014-12-22|
|R105MO44HU33PJ|   51614073|B0045DDN1G|     334117380| 2015-08-15|
|R10685DHJNVEZJ|   15280931|B00XB1H0EY|     215787563| 2015-07-27|
|R106BUZ44UVDHE|   10755897|B000PU34TO|     156625754| 2012-12

In [66]:
review_id_df.count()

2643619

In [52]:
products_df=data_df.select(["product_id", "product_title"])
products_df.count()

2643619

In [57]:
products_df=products_df.select(["product_id", "product_title"]).dropDuplicates(subset = ['product_id'])
products_df.count()

239341

In [61]:
df1.filter(df1["product_id"] == "B00025YUWC").show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00025YUWC|Blue Ribbon Natur...|
+----------+--------------------+



In [18]:
distinct_df.filter(distinct_df["product_id"] == "B00JWH8BJE").show()
# df3 = df2.filter(df2["total_votes"] >= 20)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JWH8BJE|Dog Harness Large...|
|B00JWH8BJE|Dog Harness Large...|
|B00JWH8BJE|Dog Harness Large...|
+----------+--------------------+



In [69]:
customers_df = data_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [74]:
customers_df.count()

1415190

In [80]:
vine_df = data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).dropDuplicates(subset = ['review_id'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101CJ0JBI9J43|          5|            6|          6|   N|
|R101P7GUI5O1WG|          5|            0|          0|   N|
|R102LL91IX39PK|          5|            0|          0|   N|
|R102YVSWWEAP5K|          5|            0|          0|   N|
|R1042WRU1WDUHY|          5|            7|          8|   N|
|R104G7VC1XPVOO|          5|            1|          1|   N|
|R10557FTU9GHY8|          5|            0|          0|   N|
|R105B7Y5ALJ45D|          5|            1|          1|   N|
|R105C41QCCFLIN|          5|            0|          0|   N|
|R105MO44HU33PJ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [81]:
vine_df.count()

2643619

**Postgres Setup : Write to AWS RDS instance**

In [78]:
# Write to db
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amznreviews.cx02tixyipyg.us-east-2.rds.amazonaws.com:5432/amznreviews_db"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}



In [68]:
# Write review_id_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)



In [62]:
 # Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [70]:

# Write customer_df to table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [82]:
# Write vine_df to table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)